<a href="https://colab.research.google.com/github/joyfulblooms/MUSPP/blob/main/Testing_HDB_Resale_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDB Resale Price Prediction


In [ ]:
# import pandas and numpy library
import pandas as pd
import numpy as np

In [ ]:
import os

from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns

import requests
import json

import math

# 1 Data Loading

In [ ]:
# check data file; updated the 2017 onwards file to be the latest dataset as of 17 Apr 2023. Decided that the 2017 onwards dataset is sufficient. 
os.listdir("data/")

FileNotFoundError: ignored

In [ ]:
# read data

data_2017 = pd.read_csv("data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")

In [ ]:
# check the first several lines of data_2017 with head method of pandas dataframe
data_2017.head()

In [ ]:
# convert the month column to datetime type
data_2017['month'] = pd.to_datetime(data_2017['month'])

In [ ]:
# check the info of data_wide
data_2017.info()

month : month and year of transaction <br/>
town : town of flat <br/>
flat_type : type of flat <br/>
block : block number of flat <br/>
street_name : street name of flat <br/>
storey_range : storey of flat <br/>
floor_area_sqm : floor area of flat in squared meter <br/>
flat_model : model of flat <br/>
lease_commence_date : date the lease started <br/>
resale_price : nominal resale price of flat <br/>
remaining_lease : remaining lease of flat in years <br/>

# 2 Data Analysis 

## 2.1 Feature viewing

Because we have a limited number of features and they all have specific meanings, we can look at the values of each feature in turn and perform basic statistics.

### 2.1.1 month

In [ ]:
data_2017.groupby(pd.Grouper(key='month', freq='M')).apply(lambda x: len(x)).plot.area()

In terms of time distribution, the sample is relatively uniform except that we see a significant dip in 2020 due to COVID. 

### 2.1.2 town

In [ ]:
# count the number of every town in the 'town' column of data_wide using value_counts method
data_2017['town'].value_counts()

In [ ]:
# Function for lollipop charts
def loll_plot(df, x, y, xlabel, xlim):
    plt.rc('axes', axisbelow=True)
    plt.grid(linestyle='--', alpha=0.4)
    # plt.hlines
    plt.hlines(y=df.index, xmin=0, xmax=df[x], color=df.color, linewidth=1)
    plt.scatter(df[x], df.index, color=df.color, s=300)
    for i, txt in enumerate(df[x]):
        plt.annotate(str(round(txt)), (txt, i), color='white', fontsize=9, ha='center', va='center')
    plt.yticks(df.index, df[y])
    plt.xticks(fontsize=12)
    plt.xlim(xlim)
    plt.xlabel(xlabel, fontsize=14)

In [ ]:
# proprocessing data for plotting
data_wide_town = data_2017.copy()
data_wide_town['year'] = pd.DatetimeIndex(data_wide_town['month']).year # extract out year
data_wide_town = data_wide_town.groupby(['town'], as_index=False).agg(
    {'resale_price': 'median'}).sort_values('resale_price', ascending=True).reset_index(drop=True)
data_wide_town['resale_price'] = round(data_wide_town['resale_price']/1000)
data_wide_town['color'] = ['#f8766d'] + ['#3c78d8']*(len(data_wide_town)-2) + ['#00ba38']


In [ ]:
#4-room flats
price_4room = data_2017[data_2017['flat_type'] == '4 ROOM'].copy()
price_4room['year'] = pd.DatetimeIndex(price_4room['month']).year # extract out year
price_4room = price_4room.groupby(['town'], as_index=False).agg(
    {'resale_price': 'median'}).sort_values('resale_price', ascending=True).reset_index(drop=True)
price_4room['resale_price'] = round(price_4room['resale_price']/1000)
price_4room['color'] = ['#f8766d'] + ['#3c78d8']*(len(price_4room)-2) + ['#00ba38']

In [ ]:
# create subplots
fig = plt.subplots(1, 2, figsize=(14, 7))

# plot lollipop graph for whole town's resale price
ax1 = plt.subplot(121)
loll_plot(data_wide_town, 'resale_price', 'town', 'Resale Price (SGD)', [100, 1000])
xlabels = ax1.get_xticks().tolist()
ax1.xaxis.set_major_locator(mticker.FixedLocator(xlabels))
ax1.set_xticklabels(['%.0f K'% x for x in xlabels])
ax1.yaxis.set_ticks_position('none')
ax1.set_title('Median Resale Price of All Flats')

# plot lollipop graph for 4 Room flats resale price
ax2 = plt.subplot(122)
loll_plot(price_4room, 'resale_price', 'town', 'Resale Price (SGD)', [100, 1000])
xlabels = ax2.get_xticks().tolist()
ax2.xaxis.set_major_locator(mticker.FixedLocator(xlabels))
ax2.set_xticklabels(['%.0f K'% x for x in xlabels])
ax2.yaxis.set_ticks_position('none')
ax2.set_title('Median Resale Price of 4 Room Flats')

plt.tight_layout()
plt.show()

We need to see both the Median Resale Prices of all the flats and the Median Resale Prices of 4 Room flats. This is because the former graph paints a slightly inaccurate picture. It shows ang mo kio as having the lowest resale price but it is mainly because the flats sold there are mostly 3room and 2room flats. The graph of the median prices of 4 room flats paint a more accurate picture and matches our intuition (central areas cost more, northern areas and non-mature estates cost less). 

### 2.1.3 flat_type

In [ ]:
data_2017['flat_type'].unique()

In [ ]:
data_2017['flat_type'].value_counts()

In [ ]:
# calculate ratio for every flat_type
flat_type_ratio = [val/ data_2017.shape[0] for val in data_2017['flat_type'].value_counts().values]
flat_type_label = data_2017['flat_type'].value_counts().keys()

In [ ]:
fig = plt.figure()
fig, ax = plt.subplots(figsize=(10, 10))
ax.pie(
    flat_type_ratio,
    explode=[0.005, 0.005, 0.005, 0.005, 0.05, 0.2, 0.5],
    labels=flat_type_label,
    autopct='%1.1f%%',
    shadow=False,
    startangle=90,
    pctdistance=1
)
ax.axis('equal')
plt.title("Pie chart of flat type")
plt.show()

In terms of flat types, 3-5ROOM is the most common. 

In [ ]:
#We will drop 1 Room Flat and Multi-Generation Flats since the counts are small and they are not the focus of our study. 
data_2017 = data_2017[(data_2017['flat_type'] != 'MULTI-GENERATION') & (data_2017['flat_type'] != '1 ROOM')]

### 2.1.4 block

In [ ]:
#data_wide['block'].value_counts()
data_2017['block'].value_counts()

### 2.1.5 street_name

In [ ]:
#data_wide['street_name'].value_counts()
data_2017['street_name'].value_counts()

### 2.1.6 storey_range

In [ ]:
#data_wide['storey_range'].value_counts()
data_2017['storey_range'].value_counts()

In [ ]:
fig = plt.figure(figsize=(12,4))
ax1 = plt.subplot(111)
#storey = data_wide.groupby('storey_range')['resale_price'].median().reset_index().sort_values(by='storey_range')
storey = data_2017.groupby('storey_range')['resale_price'].median().reset_index().sort_values(by='storey_range')
storey['storey_rank'] = storey['storey_range'].astype('category').cat.codes # label encode
sns.scatterplot(x=storey['storey_rank'], 
                  y=storey['resale_price'], 
                  size=storey['storey_rank'].astype('int')*30, 
                  color='green',
                  edgecolors='w', 
                  alpha=0.5, legend=False, ax=ax1)
ylabels = ax1.get_yticks().tolist()
ax1.yaxis.set_major_locator(mticker.FixedLocator(ylabels))
ax1.set_yticklabels(['%.0f K'% (x/1000) for x in ylabels])
xlabels = ax1.get_xticks().tolist()
print(xlabels)
ax1.xaxis.set_major_locator(mticker.FixedLocator(xlabels))
ax1.set_xticklabels([''] + list(storey['storey_range'].iloc[[0, 5, 10, 15, 20, 24]]) + [''])
ax1.set_ylim([280000,1200000]), ax1.set_ylabel('Resale Price SGD ($)', size=15), ax1.set_xlabel('Storey', size=15)
ax1.set_title('The impact of storey range on resale price', size=15)

The higher the storey, the higher the general housing price. This matches our intuition. 

### 2.1.7 floor_area_sqm

In [ ]:
#data_wide['floor_area_sqm'].value_counts()
data_2017['floor_area_sqm'].value_counts()

In [ ]:
# create a histogram plot of the floor_area_sqm column with bins=50 and edgecolor='black' using plt.hist
plt.hist(data_2017['floor_area_sqm'], bins=50, edgecolor ='black')
plt.title('Distribution of HDB Floor Area')
plt.show()
display(data_2017[data_2017['floor_area_sqm'] > 200]['flat_model'].value_counts())

The housing area is concentrated around 100. This matches our intuition as most flats sold are 4Room or 5Room flats. 

### 2.1.8 flat_model

In [ ]:
#data_wide['flat_model'].value_counts()
data_2017['flat_model'].value_counts()

In [ ]:
#There are many flat_models and it is best to reduce it through recategorisation
replace_values = {'Premium Maisonette':'Maisonette', 'Model A-Maisonette':'Maisonette','Improved-Maisonette':'Maisonette','Terrace':'Special', 'Adjoined flat':'Special', 
                'DBSS':'Special', 'Premium Apartment Loft':'Special','3Gen':'Special', 'Model A2':'Model A', 'Premium Apartment':'Apartment', 'Improved':'Standard', 'Simplified':'Model A', '2-room':'Standard Small', 'Type S1':'Standard Small', 'Type S2':'Standard Small'}
data_2017 = data_2017.replace({'flat_model': replace_values})

In [ ]:
data_2017['flat_model'].value_counts()

We have reduced the number of categories of flats.

### 2.1.9 lease_commence_date

In [ ]:
#data_wide['lease_commence_date'].value_counts()
data_2017['lease_commence_date'].value_counts()

In [ ]:
#bins = data_wide['lease_commence_date'].max() - data_wide['lease_commence_date'].min()
#plt.hist(data_wide['lease_commence_date'], bins=bins, edgecolor='black')
bins = data_2017['lease_commence_date'].max() - data_2017['lease_commence_date'].min()
plt.hist(data_2017['lease_commence_date'], bins=bins, edgecolor='black')
plt.title('Distribution of Lease Commence Year')
plt.show()

### 2.1.10 resale_price

In [ ]:
#data_wide['resale_price']
data_2017['resale_price']

In [ ]:
#price_range = pd.cut(data_wide['resale_price'], bins=10).value_counts().items()
price_range = pd.cut(data_2017['resale_price'], bins=10).value_counts().items()

In [ ]:
#price_range = pd.cut(data_wide['resale_price'], bins=10).value_counts().items()
price_range = pd.cut(data_2017['resale_price'], bins=10).value_counts().items()
nums = []
xlabels = []
for key, val in price_range:
    xlabels.append(key)
    nums.append(val)

Partition house prices equidistant to see the number of samples for different price ranges. The intuitive conclusion is that the lower the price, the more samples.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.bar(np.arange(1, len(nums)+1), nums)
plt.xticks(np.arange(1, len(nums)+1), xlabels, rotation=-90)
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
#ax.hist(data_wide['resale_price'], 20)
ax.hist(data_2017['resale_price'], 20)
plt.show()

The histogram effect without partitioning is also consistent. The sample size is mainly concentrated in the low price segment.

### 2.1.11 remaining_lease

In [ ]:
#data_wide['remaining_lease'].value_counts()
data_2017['remaining_lease'].value_counts()

## 2.2 Data cleaning

### 2.2.1 value substitution

Making all categorical fields lowercase helps combine values that have the same meaning but different letter cases.

In [ ]:
#data_wide_c = data_wide.copy() # make a copy of data_wide for further processing
data_wide_c = data_2017.copy() # make a copy of data_wide for further processing
for var in ['town', 'flat_type', 'block', 'street_name', 'storey_range', 'flat_model']:
    data_wide_c[var] = data_wide_c[var].str.lower()

In [ ]:
data_wide_c['flat_type'].value_counts()

In [ ]:
data_wide_c['flat_model'].value_counts()

### 2.2.2 Inflation Adjustment Using CPI

Prices in different years will be affected by inflation, and 2019 will be used as a baseline to correct the impact of inflation to more accurately compare price changes. 

In [ ]:
# Compute Resale Price Adjusted for Inflation Using Consumer Price Index for Housing & Utilities
# https://github.com/teyang-lau/HDB_Resale_Prices/blob/main/Data/CPI.csv 
# click raw -> right click -> save as
#I have included Mar 2023 and Apr 2023 CPI as the same as Feb 2023 as the runrate is similar and the monetary policies have not changed in those months so it is reasonable to assume that the rate is similar to Feb 2023. 
cpi = pd.read_csv("data/CPI_2.csv")

In [ ]:
cpi.shape

In [ ]:
cpi.dtypes

In [ ]:
cpi

In [ ]:
cpi['month'] = pd.to_datetime(cpi['month'], format='%Y %b', errors='coerce') # to datetime
data_wide_c['month'] = pd.to_datetime(data_wide_c['month'], format='%Y-%m')
# merge data_wide_c and cpi on the column month and how='left' into a new data_wide_c
data_wide_c = data_wide_c.merge(cpi, on='month', how='left')
# https://people.duke.edu/~rnau/411infla.htm
data_wide_c['real_price'] = (data_wide_c['resale_price'] / data_wide_c['cpi']) * 100 

In [ ]:
data_wide_c

In [ ]:
# plot the unadjusted and adjusted median price for each year
# Unadjusted
fig = plt.figure(figsize=(14,4.5))
fig.suptitle('Median HDB Resale Prices Over the Years', fontsize=18)
ax1 = fig.add_subplot(121)
data_wide_c.groupby('month')[['resale_price']].median().plot(ax=ax1, color='green', legend=None)
ax1.set_xlabel('Date')
ax1.set_ylabel('Resale Price in SGD ($)')
ax1.set_ylim(0, 700000)
ax1.set_title('Unadjusted for Inflation', size=15)

# Adjusted
ax2 = fig.add_subplot(122)
data_wide_c.groupby('month')[['real_price']].median().plot(ax=ax2, color='blue', legend=None)
ax2.set_xlabel('Date')
ax2.set_ylabel('Resale Price in SGD ($)')
ax2.set_ylim(0, 700000)
ax2.set_title('Adjusted for Inflation',size=15)
plt.tight_layout() 
plt.show()

### 2.2.3 Convert remaining_lease to number of years

In [ ]:
data_wide_c['remaining_lease'].unique()[:100]

In [ ]:
# write a function to normalize lease year
def getYears(text):
    if isinstance(text, str):
        yearmonth = [int(s) for s in text.split() if s.isdigit()]
        if len(yearmonth) > 1: # if there's year and month
            years = yearmonth[0] + (yearmonth[1]/12)
        else: # if only year
            years = yearmonth[0]
        return years
    else: # if int
        return text

data_wide_c['remaining_lease_year'] = data_wide_c['remaining_lease'].apply(lambda x: getYears(x))

In [ ]:
data_wide_c['remaining_lease_year'].hist(bins=50)
plt.title('Distribution of Remaining Lease for 2017-2023 Data')

The output makes sense as typically, many owners sell their homes at the 5 years mark. There are also interesting peaks at the 35 year mark and the 20 year mark. 

### 2.2.4 missing values

In [ ]:
data_wide_c.isnull().mean()

There are no missing values.

In [ ]:
#Change lease commence date to datetime
data_wide_c['lease_commence_date'] = pd.to_datetime(data_wide_c['lease_commence_date'], format='%Y')

## 2.3 EDA

### 2.3.1 time/date

Features are derived by transaction date, start lease date, and remaining lease duration. <br/>
Includes the number of years since the initial lease date; Total lease term, etc.

In [ ]:
# The number of years from the date of the transaction to the start of the lease.
data_wide_c['year_of_transaction_from_lease'] = (pd.to_datetime(data_wide_c['month'].map(str))-
                                                 pd.to_datetime(data_wide_c['lease_commence_date'].map(str))).dt.days/365

In [ ]:
data_wide_c['year_of_transaction_from_lease']

In [ ]:
data_wide_c['year_of_transaction_from_lease'].hist(bins=50)

In [ ]:
# Lease years plus remaining lease years = total lease years.
data_wide_c['whole_years_from_lease'] = data_wide_c['year_of_transaction_from_lease'] + data_wide_c['remaining_lease_year']

In [ ]:
data_wide_c['whole_years_from_lease'].hist(bins=50)

This meets our expectations as the typical lease for HDB flats is 99 years.

### 2.3.2 coordinates

The latitude and longitude coordinates corresponding to each block can be obtained through the open source interface. After obtaining the latitude and longitude coordinates, the location can be intuitively marked out and the area in the sample set can be observed. And can use longitude and latitude coordinates for clustering and other operations, through the distance to judge the house price.

In [ ]:
## Function for getting postal code, geo coordinates of addresses, and write to a csv file
def find_postal(lst, filename):
    '''With the block number and street name, get the full address of the hdb flat,
    including the postal code, geogaphical coordinates (lat/long)'''
    
    for index,add in enumerate(lst):
        # Do not need to change the URL
        url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ add
        if index % 100 == 0:
            print(index,url)
        
        # Retrieve information from website
        response = requests.get(url)
        try:
            data = json.loads(response.text) 
        except ValueError:
            print('JSONDecodeError')
            pass
    
        temp_df = pd.DataFrame.from_dict(data["results"])
        # The "add" is the address that was used to search in the website
        temp_df["address"] = add
        
        # Create the file with the first row that is read in 
        if index == 0:
            file = temp_df
        else:
            file = file.append(temp_df)
    file.to_csv(filename + '.csv')

In [ ]:
wide_block_street = data_wide_c[['block', 'street_name']]
address= (wide_block_street['block'].map(lambda x: str(x).upper()) + ' ' 
                                + wide_block_street['street_name'].map(lambda x: str(x).upper()))
wide_block_street.loc[:, 'address'] = address
all_address = list(wide_block_street['address'])
unique_address = list(set(all_address))

print('Unique addresses:', len(unique_address))

In [ ]:
# check the first five address in the list unique_address
for i in range(5):
    print (unique_address[i])

In [ ]:
# this would run for nearly an hour
# so when the flat_coordinates.csv file have been created once, you can comment this code out in order not to run it again.
#find_postal(unique_address, 'data/flat_coordinates_2')

In [ ]:
flat_coordinates = pd.read_csv("data/flat_coordinates_2.csv")
flat_coordinates.head()

In [ ]:
flat_coordinates = flat_coordinates[['address', 'LATITUDE', 'LONGITUDE']]
flat_coordinates.head()

In [ ]:
data_wide_c['address'] = wide_block_street['address']
data_wide_c2 = pd.merge(data_wide_c, flat_coordinates, on=['address'], how='left')

In [ ]:
data_wide_c2['LATITUDE'].isnull().mean(), data_wide_c2['LONGITUDE'].isnull().mean()

There are no null values

In [ ]:
data_wide_c2['street_name'].value_counts().describe([i/20 for i in range(20)])

In [ ]:
street_name_static = pd.DataFrame(data_wide_c2['street_name'].value_counts().items(), columns=['street_name', 'num'])
street_name_most = list(street_name_static[street_name_static['num']>1500]['street_name']) #I changed the listing of the number to be more than 1500 instead 10,000

In [ ]:
print(street_name_most)

In [ ]:
# plot location
data_wide_c2_street_most = data_wide_c2[data_wide_c2['street_name'].isin(street_name_most)] #create dataframe to filter just the rows where the street_name column is in the list of street_name_most 
data_wide_c2_street_most = data_wide_c2_street_most.dropna(subset=['LONGITUDE', 'LATITUDE', 'street_name'], how='any') #drop values which are missing in either the columns
color_dict = dict(zip(data_wide_c2_street_most['street_name'], data_wide_c2_street_most['street_name'].astype('category').cat.codes)) #dict where street names and values are integer codes tt rep colors for each street.
color_dict = sorted(color_dict.items(), key=lambda x: x[1]) #sort dict by values in ascending order and return a list of tuples. 
print(color_dict)  # Correspondence between street names and color numbers
scatter=plt.scatter(data_wide_c2_street_most['LONGITUDE'], data_wide_c2_street_most['LATITUDE'], s=1, 
                    c=data_wide_c2_street_most['street_name'].astype('category').cat.codes)
plt.legend(handles=scatter.legend_elements()[0],
          labels=[x[0] for x in color_dict])
plt.show()

### 2.3.3 one-hot encoding

All categorical variables are one-hot encoded.

In [ ]:
data_wide_c3 = data_wide_c2.copy()

In [ ]:
data_wide_c3.columns

In [ ]:
#Create a new column called region to reduce the dimensionality of the areas.
d_region={'ang mo kio':'North East Mature','bedok':'East Mature','bishan':'Central Mature','bukit batok':'West Non-Mature','bukit merah':'Central Mature','bukit panjang':'West Non-Mature','bukit timah':'Central Mature','central area':'Central Mature','choa chu kang':'West Non-Mature','clementi':'West Mature','geylang':'Central Mature','hougang':'North East Non-Mature','jurong east':'West Non-Mature','jurong west':'West Non-Mature','kallang/whampoa':'Central Mature','marine parade':'Central Mature','pasir ris':'East Mature','punggol':'North East Non-Mature','queenstown':'Central Mature','sembawang':'North Non-Mature','sengkang':'North East Non-Mature','serangoon':'North East Mature','tampines':'East Mature','toa payoh':'Central Mature','woodlands':'North Non-Mature','yishun':'North Non-Mature'}
data_wide_c3['region'] = data_wide_c3['town'].map(d_region)

In [ ]:
data_wide_c3['region'].isnull().mean() #Check for null values

In [ ]:
# Check if the 'region' column contains the value 'Central Mature'
if 'Central Mature' in data_wide_c3['region'].values:
    print("The 'region' column contains 'Central Mature' value.")
else:
    print("The 'region' column does not contain 'Central Mature' value.")

In [ ]:
dummy_res = []
for var in ['region', 'flat_type', 'storey_range', 'flat_model']:
    dummy_data = pd.get_dummies(data_wide_c3[var], prefix=var, dummy_na = False, drop_first = False)
    dummy_res.append(dummy_data)

In [ ]:
dummy_res[0].head()

In [ ]:
dummy_wide = pd.concat(dummy_res, axis=1)

In [ ]:
data_wide_c3 = pd.concat([data_wide_c3, dummy_wide], axis=1)

In [ ]:
if "region_Central Mature" in data_wide_c3.columns:
    print("The column 'region Central Mature' is present in the DataFrame.")
else:
    print("The column 'region Central Mature' is not present in the DataFrame.")

In [ ]:
data_wide_c3.tail()

In [ ]:
data_wide_c3['region'].isnull().sum()

In [ ]:
data_wide_c3.groupby('region')['real_price'].median()

We can see that HDB in the central region cost the most followed by the East region. Strangely the North East Non-Mature estates also look like they are priced quite highly. This could be due to a large number of flats sold in the North East Non-Mature region - Yishun - where the lease left is high that impacts the price. 

### 2.3.4 label encoding

In [ ]:
storey_dicts = sorted(list(data_wide_c3['storey_range'].unique()))
{storey_dicts[i]:i for i in range(len(storey_dicts))}

In [ ]:
data_wide_c3['storey_range_label'] = data_wide_c3['storey_range'].replace({storey_dicts[i]:i for i in range(len(storey_dicts))}).astype('category').cat.codes

Include dataset on amenities

In [ ]:
#Distance to the nearest amenities
flat_amenities = pd.read_csv('data/flat_amenities.csv')

# Merge amenities data to flat data. Realised that we need all the names of the block and street name to be in str and upper case or it will not be merged correctly. 
data_wide_c3['flat'] = data_wide_c3['block'].str.upper() + ' ' + data_wide_c3['street_name'].str.upper()
data_wide_c3 = data_wide_c3.merge(flat_amenities, on='flat', how='left')

In [ ]:
#Check if they are correctly merged. 
null_counts = data_wide_c3.isnull().sum()
print(null_counts)

In [ ]:
# get median info of each town
tmp = data_wide_c3.groupby('town')[['park_dist','num_park_2km','mall_dist','num_mall_2km','mrt_dist','num_mrt_2km','real_price']].median().reset_index()

In [ ]:
# scatterplot for median price of each town against nearest distance from each amenity

p=sns.pairplot(tmp, x_vars=["park_dist", "mall_dist", "mrt_dist"], y_vars=["real_price"], height=3, aspect=1, kind="reg", plot_kws=dict(scatter_kws=dict(s=40)))
axes=p.axes
ylabels = ['{:,.0f}'.format(x) + 'K' for x in axes[0,0].get_yticks()/1000]
axes[0,0].set_yticklabels(ylabels), axes[0,0].set_ylabel('Resale Price SGD ($)', size=10)
axes[0,0].set_xlabel('Distance From Park (km)', size=10), axes[0,1].set_xlabel('Distance From Mall (km)', size=10),axes[0,2].set_xlabel('Distance From MRT (km)', size=10)
plt.suptitle('Resale Price (Median of Each Town) VS Distance from Nearest Amenities (Median of Each Town)')
plt.tight_layout(pad=0, rect=[0, 0, 0.9, 0.9])
plt.show()

We can see that distance from park, mall and MRT have a negative correlation with the price of the HDB. 

In [ ]:
# scatterplot for median price of each town against number of amenities

p=sns.pairplot(tmp, x_vars=["num_park_2km", "num_mall_2km", "num_mrt_2km"], y_vars=["real_price"], height=3, aspect=1, kind="reg", plot_kws=dict(scatter_kws=dict(s=40)))
axes=p.axes
ylabels = ['{:,.0f}'.format(x) + 'K' for x in axes[0,0].get_yticks()/1000]
axes[0,0].set_yticklabels(ylabels), axes[0,0].set_ylabel('Resale Price SGD ($)', size=10)
axes[0,0].set_xlabel('Number of Parks', size=10), axes[0,1].set_xlabel('Number of Malls', size=10)
axes[0,2].set_xlabel('Number of MRTs', size=10)
plt.suptitle('Resale Price (Median of Each Town) VS Number of Amenities in 2km Radius (Median of Each Town)')
plt.tight_layout(pad=0, rect=[0, 0, 0.9, 0.9])
plt.show()

We can see that the number of parks, malls and MRTs are also positively related to the price of the HDB flat. 

### 2.3.4 feature dtypes

In [ ]:
data_wide_c3.dtypes.apply(lambda x:x.kind).value_counts()

In [ ]:
# select x variables
model_columns = list(data_wide_c3.columns[data_wide_c3.dtypes.apply(lambda x: x.kind).isin(['f', 'u', 'i'])])
model_columns.remove('real_price')
model_columns.remove('resale_price')
model_columns.remove('cpi')

In [ ]:
len(model_columns)

In [ ]:
y_label = 'real_price'

In [ ]:
list(data_wide_c3.columns[data_wide_c3.dtypes.apply(lambda x: x.kind).isin(['f', 'i'])])

In [ ]:
data_wide_c3.info()

Check for multicolinearity to check if the indpt variables are highly correlated. We are interested in the importance of features by looking at the coefficients of the linear regression model hence we have to correct for it. VIF greater than 10 is cause for concern. Average VIF is greater than 1 then regression may be biased. Tolerance below 0.1 is a problem. 

In [ ]:
#Correlation heatmap
fig= plt.figure(figsize=(10,10))
ax= sns.heatmap(data_wide_c3.select_dtypes(include=['int64','float64']).corr(), annot=True, fmt='.2g', vmin=-1, vmax=1, center=0, cmap='coolwarm_r', linecolor='black', linewidths=1, annot_kws={"size":7})
plt.xticks(rotation=45, ha='right')
plt.title('Correlation Heatmap')
fig.show()

From the above heatmap, we can see that unsurprisingly, remaining_lease_year, year_of_transaction_from_lease, whole_years_from_lease and  are correlated. Since our interest is to find out the relation with resale price, we will choose the remaining_lease_year since its correlation coefficient to the real_price is the highest as compared to the rest.

In [ ]:
#create new dataset which reduces the lease year to just lease_commence_date. The cpi and the latitude and longitude are also not needed at this stage. We also want to focus on the real price instead of the resale price and we will drop the resale price as well. 
data_wide_d = data_wide_c3.copy()
data_wide_d.drop(['year_of_transaction_from_lease', 'whole_years_from_lease', 'cpi', 'LATITUDE', 'LONGITUDE', 'resale_price'], axis=1, inplace=True)
data_wide_d.columns

Let us plot the heatmap again to see more clearly how the factors are correlated. 

In [ ]:
#Correlation heatmap
fig= plt.figure(figsize=(10,10))
ax= sns.heatmap(data_wide_d.select_dtypes(include=['int64','float64']).corr(), annot=True, fmt='.2g', vmin=-1, vmax=1, center=0, cmap='coolwarm_r', linecolor='black', linewidths=1, annot_kws={"size":7})
plt.xticks(rotation=45, ha='right')
plt.title('Correlation Heatmap')
fig.show()

In [ ]:
#Import library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):
    #New drop rows with missing or infinite values
    X= X.replace([np.inf, -np.inf],np.nan).dropna()
    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['tolerance'] = 1/vif.VIF
    vif['meanVIF'] = vif.VIF.mean()

    return(vif)

calc_vif(data_wide_d.drop(['real_price'],axis=1).select_dtypes(include=['int64','float64']))

The VIF is very high and it is not ideal. There is multi-colinearity in the data but these features are important and hence we choose to leave them in. 

In [ ]:
#We will try to drop some more features to see if the VIF can be reduced. 
calc_vif(data_wide_d.drop(['real_price', 'num_park_2km', 'num_mall_2km', 'num_mrt_2km'],axis=1).select_dtypes(include=['int64','float64']))

This is a better VIF but the floor_area and the lease_commence_date are still very high.

In [ ]:
#We will try to drop some more features to see if the VIF can be reduced. 
calc_vif(data_wide_d.drop(['real_price', 'num_mall_2km', 'floor_area_sqm', 'remaining_lease_year'],axis=1).select_dtypes(include=['int64','float64']))

If we drop floor_area_sqm and remaining_lease_year, we see that the VIF and tolerance are alot more acceptable. VIF is below 5 and tolerance is above 0.2. 

In [ ]:
#Checking for normality
lr_df=data_wide_d.select_dtypes(include=['int64','float64'])
lr_df.hist(bins=50, figsize=(15,10), grid=False, edgecolor='black')
plt.tight_layout(pad=0, rect=[0,0,0.9,0.9])
plt.show()

Many variables do not follow a normal distribution, and most of the distances variables have some outliers. 

In [ ]:
#We will log the real price to make it more normally distributed.
# plot qqplot before and after log transformation
from statsmodels.api import qqplot
fig, ((ax1,ax2), (ax3,ax4)) = plt.subplots(2,2,figsize=(5,5))

ax1.hist(lr_df['real_price'], bins=50, edgecolor='black')
qqplot(lr_df['real_price'], line='s', ax=ax2)
ax3.hist(np.log(lr_df['real_price']), bins=50, edgecolor='black')
qqplot(np.log(lr_df['real_price']), line='s', ax=ax4)
plt.suptitle('Real Price Normality Before (Top) & After (Bottom) Logging')
plt.tight_layout(pad=0, rect=[0, 0, 0.9, 0.9])
plt.show()

Logging the real price produces a better result. 

# 3 model

## 3.1 Linear Regression

In [ ]:
sub_model_columns = ['floor_area_sqm','remaining_lease_year','storey_range_label', 'region_Central Mature', 'region_East Mature','region_West Mature', 'region_West Non-Mature', 'region_North Non-Mature', 'region_North East Mature', 'region_North East Non-Mature','park_dist','mrt_dist','mall_dist','num_park_2km','num_mall_2km','num_mrt_2km'
]

In [ ]:
# Correlation heatmap
fig = plt.figure(figsize=(10,10))
ax = sns.heatmap(data_wide_d[sub_model_columns].sample(10000).corr('spearman'), annot = True, fmt='.2g', 
    vmin=-1, vmax=1, center= 0, cmap= 'coolwarm_r', linecolor='black', linewidth=1, annot_kws={"size": 7})
#ax.set_ylim(0 ,5)
plt.xticks(rotation=45, ha='right')
plt.title('Correlation Heatmap')
fig.show()

In [ ]:
lr_df = data_wide_d.copy()

In [ ]:
lr_df.head()

We need to scale the data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# fit to continuous columns and transform
scaled_columns = ['floor_area_sqm','remaining_lease_year','storey_range_label', 'region_Central Mature', 'region_East Mature','region_West Mature', 'region_West Non-Mature', 'region_North Non-Mature', 'region_North East Mature', 'region_North East Non-Mature','park_dist','mrt_dist','mall_dist','num_park_2km','num_mall_2km','num_mrt_2km'
]
scaler.fit(lr_df[scaled_columns])
scaled_columns = pd.DataFrame(scaler.transform(lr_df[scaled_columns]), index=lr_df.index, columns=scaled_columns)

# separate unscaled features
unscaled_columns = lr_df.drop(scaled_columns, axis=1)

# concatenate scaled and unscaled features
lr_df = pd.concat([scaled_columns,unscaled_columns], axis=1)

In [ ]:
lr_df.head()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


We need to scale the data

In [ ]:
y_label='real_price'

In [ ]:
# split data set into train set and test set with x and y separate
# four dataframe generated:train_y, test_x, test_y, train_x, you should arrange them in proper order
train_x, test_x, train_y, test_y = train_test_split(lr_df[sub_model_columns], lr_df[y_label], 
    test_size=0.3, random_state=0)

In [ ]:
# create a regressor named lin_reg with LinearRegression and fit it with train_x and np.log(train_y)
lin_reg = LinearRegression()
lin_reg.fit(train_x, np.log(train_y)) #we need to log the train_y data as it is not normally distributed and we want to improve the linearity. 

In [ ]:
lin_reg_pred = lin_reg.predict(test_x)

In [ ]:
#Include Lasso and Ridge Regularization [New] 
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import r2_score, mean_squared_error

#Create Lasso model [New]
clf_L1= Lasso(alpha=0.3)
clf_L1.fit(train_x, np.log(train_y))

y_pred_L1= clf_L1.predict(test_x)

print(f'Coefficients: {clf_L1.coef_}')
print(f'Intercept: {clf_L1.intercept_}')
print(f'R^2 score for train set: {clf_L1.score(train_x, np.log(train_y))}')
print(f'R^2 score for test set: {clf_L1.score(test_x, np.log(test_y))}')

In [ ]:
#Create Ridge model [New]
clf_L2= Ridge(alpha=0.3)
clf_L2.fit(train_x, np.log(train_y))

y_pred_L2= clf_L2.predict(test_x)

print(f'Coefficients: {clf_L2.coef_}')
print(f'Intercept: {clf_L2.intercept_}')
print(f'R^2 score for train set: {clf_L2.score(train_x, np.log(train_y))}')
print(f'R^2 score for test set: {clf_L2.score(test_x, np.log(test_y))}')

In [ ]:
#Linear Regression Coeff, intercept
print(f'Coefficients: {lin_reg.coef_}')
print(f'Intercept: {lin_reg.intercept_}')
print(f'R^2 score for train set: {lin_reg.score(train_x, np.log(train_y))}')
print(f'R^2 score for test set: {lin_reg.score(test_x, np.log(test_y))}')

In [ ]:
#Print MSE for all three models to compare the performance - output shows that ridge and linear regression are better performers than lasso 
Lasso_mse = mean_squared_error(np.log(test_y), y_pred_L1)
Ridge_mse = mean_squared_error(np.log(test_y), y_pred_L2)
LinearReg_mse = mean_squared_error(np.log(test_y), lin_reg_pred)

print(f'Lasso MSE: {Lasso_mse:.2f}')
print(f'Ridge MSE: {Ridge_mse:.2f}')
print(f'Linear Regression MSE: {LinearReg_mse:.2f}')

The best output is the RSE and linear regression model as they have a low MSE and a high R sqaured score. 

Scatterplot for linear regression model R2

In [ ]:
# scatterplot of y (observed) and yhat (predicted) for train set using the Ridge model. 
plt.style.use('default')
fig = plt.figure(figsize=(5,3))
ax = sns.scatterplot(x=np.log(train_y), y=lin_reg.predict(train_x), edgecolors='w', alpha=0.9, s=8)
ax.set_xlabel('Observed (ln)')#, ax.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in ax.get_xticks()/1000])
ax.set_ylabel('Predicted (ln)')#, ax.set_yticklabels(['{:,.0f}'.format(x) + 'K' for x in ax.get_yticks()/1000])
ax.annotate('Adjusted R\u00b2: ' + str(format(round(lin_reg.score(train_x, np.log(train_y)),4),'.2f')), xy=(0, 1), xytext=(25, -25),
    xycoords='axes fraction', textcoords='offset points', fontsize=12)
plt.show()

Scatterplot for Ridge Model R2

In [ ]:
# scatterplot of y (observed) and yhat (predicted) for train set using the Ridge model. 
plt.style.use('default')
fig = plt.figure(figsize=(5,3))
ax = sns.scatterplot(x=np.log(train_y), y=clf_L2.predict(train_x), edgecolors='w', alpha=0.9, s=8)
ax.set_xlabel('Observed (ln)')#, ax.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in ax.get_xticks()/1000])
ax.set_ylabel('Predicted (ln)')#, ax.set_yticklabels(['{:,.0f}'.format(x) + 'K' for x in ax.get_yticks()/1000])
ax.annotate('Adjusted R\u00b2: ' + str(format(round(clf_L2.score(train_x, np.log(train_y)),4),'.2f')), xy=(0, 1), xytext=(25, -25),
    xycoords='axes fraction', textcoords='offset points', fontsize=12)
plt.show()

Summary stats for test set, OLS regression

In [ ]:
# statsmodel method, which gives more info
import statsmodels.api as sm
# alternate way using statistical formula, which does not require dummy coding manually
# https://stackoverflow.com/questions/50733014/linear-regression-with-dummy-categorical-variables
# https://stackoverflow.com/questions/34007308/linear-regression-analysis-with-string-categorical-features-variables

X_constant = sm.add_constant(test_x)
lin_reg = sm.OLS(np.log(test_y),X_constant).fit()
lin_reg.summary()

Summary stats for train set, OLS

In [ ]:
X_constant2 = sm.add_constant(train_x)
lin_reg2 = sm.OLS(np.log(train_y),X_constant2).fit()
lin_reg2.summary()

Check for homoscedasticity and normality of residuals

In [ ]:
# Homoscedasticity and Normality of Residuals
pred = lin_reg.predict()
resids = lin_reg.resid
resids_studentized = lin_reg.get_influence().resid_studentized_internal

fig = plt.figure(figsize=(10,3))

ax1 = plt.subplot(121)
sns.scatterplot(x=pred, y=resids_studentized, edgecolors='w', alpha=0.9, s=8)
ax1.set_xlabel('Predicted Values')
ax1.set_ylabel('Studentized Residuals')

ax2 = plt.subplot(122)
sns.distplot(resids_studentized, norm_hist=True, hist_kws=dict(edgecolor='w'))
ax2.set_xlabel('Studentized Residual')

plt.show()

Homoscedaticity appears to be satisfied. The residuals are normally distributed around 0, satisfying the linearity and normality assumptions of the linear model.

In [ ]:
lr_df.columns

## 3.2 random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.stats import spearmanr, pearsonr
from sklearn.tree import plot_tree
from sklearn.datasets import make_regression

In [ ]:
sub_model_columns = ['floor_area_sqm','remaining_lease_year','region_Central Mature', 'region_East Mature','region_West Mature', 'region_West Non-Mature', 'region_North Non-Mature', 'region_North East Mature', 'region_North East Non-Mature','park_dist','mrt_dist','mall_dist','num_park_2km','num_mall_2km','num_mrt_2km', 'flat_model_apartment', 'flat_model_maisonette',
       'flat_model_model a', 'flat_model_new generation', 'flat_model_special',
       'flat_model_standard', 'flat_model_standard small', 'storey_range_label'
]

In [ ]:
train_x2, test_x2, train_y2, test_y2 = train_test_split(lr_df[sub_model_columns], lr_df[y_label], test_size=0.3, random_state=0)

### 3.2.1 filter features

In [ ]:
# If you have too much data, you will run out of memory, 
# so start with a small amount of data to filter the features. 
train_x2_for_filter = train_x2.sample(10000, random_state=0)
train_y2_for_filter = train_y2.loc[train_x2_for_filter.index]

In [ ]:
# Validation using out-of-bag method
# create a RandomForestRegressor named rf with n_estimators=100, oob_score=True, random_state=0
rf_for_filter = RandomForestRegressor(n_estimators=100, oob_score=True, random_state=0)
rf_for_filter.fit(train_x2_for_filter, train_y2_for_filter)

print(f'Out-of-bag R\u00b2 score estimate: {rf_for_filter.oob_score_:>5.3}')

In [ ]:
feature_importance_frame = pd.DataFrame(zip(sub_model_columns, rf_for_filter.feature_importances_), columns=['feature', 'importance'])
feature_importance_frame.head()

In [ ]:
# select top 15 important features
feature_importance_frame = feature_importance_frame.sort_values(by='importance', ascending=False).reset_index(drop=True)
selected = list(feature_importance_frame.iloc[:15]['feature'])

In [ ]:
print(feature_importance_frame.iloc[:15])

In [ ]:
print(selected)

### 3.2.2 fit model using filtered features

In [ ]:
rf = RandomForestRegressor(n_estimators=100, oob_score=True, random_state=0, n_jobs=-1)
#train the rf with train_x2 and train_y2 using only selected features
rf.fit(train_x2[selected], train_y2)

predicted_train = rf.predict(train_x2[selected])
predicted_test = rf.predict(test_x2[selected])


In [ ]:
# get the prediction of rf on train_x2 and test_x2, remember to use only selected features
predicted_train = rf.predict(train_x2[selected])
predicted_test = rf.predict(test_x2[selected])

In [ ]:
train_score = r2_score(train_y2, predicted_train)
spearman = spearmanr(train_y2, predicted_train)
pearson = pearsonr(train_y2, predicted_train)
oob_mae = mean_absolute_error(train_y2, predicted_train)

print(f'Train data R\u00b2 score: {train_score:>5.3}')
print(f'Train data Spearman correlation: {spearman[0]:.3}')
print(f'Train data Pearson correlation: {pearson[0]:.3}')
print(f'Train data Mean Absolute Error: {round(oob_mae)}')

In [ ]:
test_score = r2_score(test_y2, predicted_test)
spearman = spearmanr(test_y2, predicted_test)
pearson = pearsonr(test_y2, predicted_test)
oob_mae = mean_absolute_error(test_y2, predicted_test)

print(f'Test data R\u00b2 score: {test_score:>5.3}')
print(f'Test data Spearman correlation: {spearman[0]:.3}')
print(f'Test data Pearson correlation: {pearson[0]:.3}')
print(f'Test data Mean Absolute Error: {round(oob_mae)}')

In [ ]:
#K-fold cross validation
from sklearn.model_selection import GridSearchCV

# validation by k-fold cross validation with grid search for best hyperparameters
# hyperparameter values shown below are the tuned final values
param_grid = {
    'max_features': ['auto'], # max number of features considered for splitting a node
    'max_depth': [20], # max number of levels in each decision tree
    'min_samples_split': [15], # min number of data points placed in a node before the node is split
    'min_samples_leaf': [2]} # min number of data points allowed in a leaf node
rfr =GridSearchCV(RandomForestRegressor(n_estimators = 500, n_jobs=-1, random_state=0),
                        param_grid, cv=10, scoring='r2', return_train_score=True)
rfr.fit(train_x2, train_y2)
print("Best parameters set found on Cross Validation:\n\n", rfr.best_params_)
print("\nCross Validation R\u00b2 score:\n\n", rfr.best_score_.round(3))

In [ ]:
# predict and get evaluation metrics for test set

cv_predicted_test = rfr.predict(test_x2)

cv_test_score = r2_score(test_y2, cv_predicted_test)
spearman = spearmanr(test_y2, cv_predicted_test)
pearson = pearsonr(test_y2, cv_predicted_test)
cv_mae = mean_absolute_error(test_y2, cv_predicted_test)

print(f'Test data R\u00b2 score: {cv_test_score:>5.3}')
print(f'Test data Spearman correlation: {spearman[0]:.3}')
print(f'Test data Pearson correlation: {pearson[0]:.3}')
print(f'Test data Mean Absolute Error: {round(cv_mae)}')

In [ ]:
# Get the best estimator from the GridSearchCV
best_estimator = rfr.best_estimator_

# Get the feature importances
feature_importances = best_estimator.feature_importances_

# Create a dictionary of feature importances with their corresponding column names
feat_imp_dict = dict(zip(train_x2.columns, feature_importances))

# Sort the dictionary by feature importances in descending order
sorted_feat_imp = sorted(feat_imp_dict.items(), key=lambda x: x[1], reverse=True)

# Print the top 10 features with their corresponding importance values
print("Top 10 features with highest importance values:")
for i in range(10):
    print(f"{i+1}. {sorted_feat_imp[i][0]}: {sorted_feat_imp[i][1]}")

In [ ]:
#save the models rfr and rf
import pickle

models_dict = {'rfr': rfr, 'rf': rf}
with open('models.pkl', 'wb') as file:
    pickle.dump(models_dict, file)

In [ ]:
pip install shap

In [ ]:
test_y2.head(10)

SHAP Values to see the factors that contribute to the price of different tiers of HDB 

In [ ]:
#Use SHAP values. 
import shap
shap.initjs()

import matplotlib.pyplot as plt

#flat with low resale price
explainer = shap.TreeExplainer(rfr.best_estimator_)
shap_values = explainer.shap_values(test_x2.iloc[[1956]])
fig1 =shap.force_plot(explainer.expected_value[0], shap_values[0], test_x2.iloc[[1956]], matplotlib=True, show=False)


In [ ]:
fig1

In [ ]:
#flat with predicted medium resale price
explainer = shap.TreeExplainer(rfr.best_estimator_)
shap_values = explainer.shap_values(test_x2.iloc[[6]])
fig2= shap.force_plot(explainer.expected_value[0], shap_values[0], test_x2.iloc[[6]], matplotlib=True, show=False)

In [ ]:
fig2

In [ ]:
#flat with high resale price
explainer = shap.TreeExplainer(rfr.best_estimator_)
shap_values = explainer.shap_values(test_x2.iloc[[172]])
fig3 = shap.force_plot(explainer.expected_value[0], shap_values[0], test_x2.iloc[[172]], matplotlib=True, show=False)


In [ ]:
fig3

#XGBoost

In [ ]:
pip install xgboost

In [ ]:
conda install graphviz python-graphviz

In [ ]:
from xgboost import XGBRegressor

# define model (complciated model listed for reference)
XGBmodel = XGBRegressor(n_estimators=1000, max_depth=6, learning_rate=0.1,early_stopping_rounds=50)

# fit model on train
display(XGBmodel)

In [ ]:
XGBmodel.fit(train_x2, train_y2,eval_set=[(test_x2, test_y2),(train_x2, train_y2)], verbose=50)

# make a prediction
yhat = XGBmodel.predict(test_x2, ntree_limit=XGBmodel.best_iteration)
Root_mean_squared_error = np.mean((test_y2-yhat)**2)**.5


In [ ]:
# summarize prediction
print('Predicted RMSE: %.3f' % Root_mean_squared_error)

In [ ]:
xgb_test_score = r2_score(test_y2, yhat)
spearman = spearmanr(test_y2, yhat)
pearson = pearsonr(test_y2, yhat)
xgb_mae = mean_absolute_error(test_y2, yhat)

print(f'Test data R\u00b2 score: {xgb_test_score:>5.3}')
print(f'Test data Spearman correlation: {spearman[0]:.3}')
print(f'Test data Pearson correlation: {pearson[0]:.3}')
print(f'Test data Mean Absolute Error: {round(xgb_mae)}')

In [ ]:
from matplotlib.pylab import rcParams

# set up the parameters; we can right click in jupyter notebook and save directly as .png vector file
rcParams['figure.figsize'] = 400,400

from xgboost import plot_tree

plot_tree(XGBmodel, num_trees = 7)

In [ ]:
# This works well too: write a helper function to export DT in high res
# credits: https://stackoverflow.com/questions/37340474/xgb-plot-tree-font-size-python

def plot_tree(xgb_model, filename, rankdir='UT'):
    """
    Plot the tree in high resolution
    :param xgb_model: xgboost trained model
    :param filename: the pdf file where this is saved
    :param rankdir: direction of the tree: default Top-Down (UT), accepts:'LR' for left-to-right tree
    :return:
    """
    import xgboost as xgb
    import os
    gvz = xgb.to_graphviz(xgb_model, num_trees=7, rankdir=rankdir)
    _, file_extension = os.path.splitext(filename)
    format = file_extension.strip('.').lower()
    data = gvz.pipe(format=format)
    full_filename = filename
    with open(full_filename, 'wb') as f:
        f.write(data)

In [ ]:
# call the function to create the export file you want
plot_tree(XGBmodel, 'XGBoost_DT.pdf')
plot_tree(XGBmodel, 'XGBoost_DT.png')
plot_tree(XGBmodel, 'XGBoost_DT_LR.pdf', 'LR')
plot_tree(XGBmodel, 'XGBoost_DT_LR.png', 'LR')

In [ ]:
# Get feature importance scores
importance_scores = XGBmodel.feature_importances_

# Create a list of (feature name, importance score) tuples
features = list(zip(train_x2.columns, importance_scores))

# Sort the list in descending order by importance score
features.sort(key=lambda x: x[1], reverse=True)

# Print the top N features
N = 20
print(f"Top {N} most important features:")
for i in range(N):
    print(f"{i+1}. {features[i][0]} ({features[i][1]:.4f})")